In [1]:
%%time

import os
import re
import string
import pprint

import pandas as pd
import numpy as np

import gensim

Wall time: 7.35 s


In [2]:
%%time

# change working directory

print('Before chdir:', os.getcwd())

os.chdir('F:\\CG\\w2vCustom') 
         
print('After chdir:', os.getcwd())


Before chdir: C:\Users\Ravi
After chdir: F:\CG\w2vCustom
Wall time: 0 ns


In [3]:
%%time

df=pd.read_excel("wb_causecode_asgrp.xlsx");

df.head()

Wall time: 15.8 s


,account,app_name,assigned_group,causecode_lv1,causecode_lv2,causecode_lv3,incident_id,summary
0,wb,NaN,WBEI.Sharepoint,Access,Issue,Unable to access application,RTASK0033182,User cannot access Time-Off Tracker
1,wb,SAP Accounts Payable/Workflow,WBEI.MKTMGT.Siebel North America,Delete,Data Deletion,As per user requirement,RTASK0086491,"Promo1169410 ACB#236740 Deal Status ""Payment i..."
2,wb,NaN,WBEI.MIS.Alfred,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0106734,Error messge stating that the Alternate Payee ...
3,wb,SAP Netweaver Business Warehouse,WBEI.BW - Data Warehouse,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0246906,FRB Reconciliation need to be perform on 06/04...
4,wb,System,WBEI.WBCAS Adobe LiveCycle,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0286950,!! URGENT-- Forward eform 11828 to another app...


In [5]:
%%time

# Function to keep required columns in a dataframe

def keepcols(df, cols):
    '''
    df = dataframe
    cols = list of columns to keep in df 
    '''
    
    cols_keep = [cols[0], cols[1], cols[2], cols[3]]
    return df[cols_keep]


# Subset required columns
df1 = keepcols(df, ['causecode_lv1', 'causecode_lv2', 'causecode_lv3', 'summary'])

df1.head()

Wall time: 7 ms


,causecode_lv1,causecode_lv2,causecode_lv3,summary
0,Access,Issue,Unable to access application,User cannot access Time-Off Tracker
1,Delete,Data Deletion,As per user requirement,"Promo1169410 ACB#236740 Deal Status ""Payment i..."
2,Access,Approval,Access/SAP Issue/Unable to access environment,Error messge stating that the Alternate Payee ...
3,Access,Approval,Access/SAP Issue/Unable to access environment,FRB Reconciliation need to be perform on 06/04...
4,Access,Approval,Access/SAP Issue/Unable to access environment,!! URGENT-- Forward eform 11828 to another app...


In [6]:
%%time

# Check for missing vlaues after iputation in a dataframe

print("Before", df1.isnull().values.any())


# check and impute missing values in a dataframe


df1 = df1.fillna('')

print("After", df1.isnull().values.any())

Before True
After False
Wall time: 58 ms


In [7]:
%%time

import re
from gensim.parsing.preprocessing import remove_stopwords

# Preprocess text for summary
def clean_sentence(sentence, stopwords=False):
    
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    #lower case 
    sentence = sentence.lower()
    # square brackets
    sentence = re.sub('\[.*?\]', '', sentence)  
    # hyperlinks
    sentence = re.sub('https?://\S+|www\.\S+', '', sentence)
    sentence = re.sub('<.*?>+', '', sentence)
    # remove punctuation
    sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence)
    sentence = re.sub('\n', '', sentence)
    # remove year
    sentence = re.sub(r"\b(19[40][0-9]|20[0-1][0-9]|2020)\b",'',sentence)
    # remove month names
    mp = r"(\b\d{1,2}\D{0,3})?\b(?:Jan|jan(?:uary)?|Feb|feb(?:ruary)?|Mar|mar(?:ch)?|Apr|mar(?:il)?|May|may|Jun|jun(?:e)?|Jul|jul(?:y)?|Aug|aug(?:ust)?|Sep|sep(?:tember)?|Oct|oct(?:ober)?|(Nov|nov|Dec|dec)(?:ember)?)\D?(\d{1,2}(st|nd|rd|th)?)?(([,.\-\/])\D?)?((19[7-9]\d|20\d{2})|\d{2})*"
    sentence = re.sub(mp, '', sentence)
     
    # remove words containing numbers
    sentence = re.sub('\w*\d\w*', '', sentence)
    # remove numbers 
    sentence = re.sub(r'[0-9]+','',sentence)
    
    if stopwords:
        sentence = remove_stopwords(sentence)
    
    return sentence

Wall time: 0 ns


In [8]:
%%time

def get_cleaned_sentences(df, col, stopwords=True):
    cleaned_sents = []
    
    for index, row in df.iterrows():
        cleaned = clean_sentence(row[col])
        cleaned_sents.append(cleaned)
    return cleaned_sents


# Calling the function get_cleaned_text

cleaned_sentences = get_cleaned_sentences(df1, 'summary')
cleaned_sentences

Wall time: 11.1 s


['user cannot access timeoff tracker',
 '  deal status payment in progress with no payment',
 'error messge stating that the alternate payee group does not exist',
 'frb reconciliation need to be perform on  ',
 ' urgent forward eform  to another approver',
 'report migration from dev to qa',
 'disable ei batch jobs',
 'complete request  general service request',
 'hpqc access to project workday domain mis  id xvdhake',
 'help withsetting up cbs profile in wbconnect',
 'route ticket to new approver',
 'proactive breakfix  incident ticket for  monthly release into production',
 'add an approver to hierarchy',
 'change order in cbs',
 'substitute approver',
 'need  to configure drivers in order to refresh lumira on bi launchpad ',
 ' assistance with filling out ',
 'proactive emergency breakfix  incident ticket for   monthly release into production',
 'eform ',
 'list of people who have role for company code ',
 'complete request  general service request',
 'please migrate the bo objects

In [10]:
%%time
sentences = cleaned_sentences
sentence_words = [[word for word in sent.split()]
                  for sent in sentences]

filtered_words = [[w for w in lst if len(w) > 2] for lst in sentence_words]         
                  
filtered_words


Wall time: 443 ms


[['user', 'cannot', 'access', 'timeoff', 'tracker'],
 ['deal', 'status', 'payment', 'progress', 'with', 'payment'],
 ['error',
  'messge',
  'stating',
  'that',
  'the',
  'alternate',
  'payee',
  'group',
  'does',
  'not',
  'exist'],
 ['frb', 'reconciliation', 'need', 'perform'],
 ['urgent', 'forward', 'eform', 'another', 'approver'],
 ['report', 'migration', 'from', 'dev'],
 ['disable', 'batch', 'jobs'],
 ['complete', 'request', 'general', 'service', 'request'],
 ['hpqc', 'access', 'project', 'workday', 'domain', 'mis', 'xvdhake'],
 ['help', 'withsetting', 'cbs', 'profile', 'wbconnect'],
 ['route', 'ticket', 'new', 'approver'],
 ['proactive',
  'breakfix',
  'incident',
  'ticket',
  'for',
  'monthly',
  'release',
  'into',
  'production'],
 ['add', 'approver', 'hierarchy'],
 ['change', 'order', 'cbs'],
 ['substitute', 'approver'],
 ['need', 'configure', 'drivers', 'order', 'refresh', 'lumira', 'launchpad'],
 ['assistance', 'with', 'filling', 'out'],
 ['proactive',
  'emergency

In [11]:
%%time
# Creating gensim word2vec model
import multiprocessing
from gensim.models import Word2Vec

# Count no. of cores in computer
cores = multiprocessing.cpu_count()

# Model parameters
wb_word2vec_model = Word2Vec(min_count=5,
                            window=5,
                            size=300,
                            sample=6e-5,
                            alpha=0.03,
                            min_alpha=0.0007,
                            negative=20,
                            workers=cores-1)




# Build vacoubalry
wb_word2vec_model.build_vocab(filtered_words)


# Training the model
wb_word2vec_model.train(filtered_words, total_examples=wb_word2vec_model.corpus_count, epochs=30)


# summarize vocabulary
words = list(wb_word2vec_model.wv.vocab)
print(words)

['user', 'cannot', 'access', 'tracker', 'deal', 'status', 'payment', 'progress', 'with', 'error', 'stating', 'that', 'the', 'alternate', 'payee', 'group', 'does', 'not', 'exist', 'frb', 'reconciliation', 'need', 'perform', 'urgent', 'forward', 'eform', 'another', 'approver', 'report', 'migration', 'from', 'dev', 'disable', 'batch', 'jobs', 'complete', 'request', 'general', 'service', 'hpqc', 'project', 'workday', 'domain', 'mis', 'help', 'cbs', 'profile', 'wbconnect', 'route', 'ticket', 'new', 'breakfix', 'incident', 'for', 'monthly', 'release', 'into', 'production', 'add', 'hierarchy', 'change', 'order', 'substitute', 'configure', 'refresh', 'launchpad', 'assistance', 'filling', 'out', 'emergency', 'list', 'people', 'who', 'have', 'role', 'company', 'code', 'please', 'migrate', 'objects', 'master', 'srm', 'issues', 'modify', 'settings', 'support', 'teradata', 'data', 'per', 'when', 'trying', 'log', 'sharepoint', 'path', 'adobe', 'eforms', 'attention', 'required', 'form', 'applicant', 

In [12]:
%%time
# Save word2vec model
wb_word2vec_model.save("./wb_word2vec_model.model")

# Save WV
wb_word2vec_model.wv.save_word2vec_format("wb_word2vec_model.txt")


# Load the custom wb_word2vec_model 

wb_word2vec_model = gensim.models.KeyedVectors.load_word2vec_format("./wb_word2vec_model.txt", binary = False)
print("wb_word2vec_model")


wb_word2vec_model
Wall time: 3.17 s


In [17]:
%%time
# Create BOW
# Create corpora
from gensim import corpora


dictionary = corpora.Dictionary(filtered_words)

# # Creating BOW

# bow_corpus = [dictionary.doc2bow(word) for word in filtered_words]
# for sent,embedding in zip(cleaned_sentences,bow_corpus):
#     print(sent)
#     print(embedding)

Wall time: 1.3 s


In [24]:
%%time
from gensim import corpora

from gensim import corpora

dictionary = corpora.Dictionary(filtered_words)


#question_orig="do I need to learn algorithms to be a data scientist ?";
query= "network error sap"
query=clean_sentence(query,stopwords=True);
query_embedding = dictionary.doc2bow(query.split())


print("\n\n",query,"\n",query_embedding)



 network error sap 
 [(12, 1), (119, 1), (344, 1)]
Wall time: 1.3 s


In [20]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def get_cosim(query,docs):
      
    count = []
    simscore = []
    result = []

    for index,faq_embedding in enumerate(docs):
            sim=cosine_similarity(faq_embedding,query)[0][0]

            count.append(index)
            simscore.append(sim)
            result.append(sentences[index])



    dfScore = pd.DataFrame(list(zip(count, simscore, result)),
                  columns=['id','simscore', 'result'])


    dfsort = dfScore.sort_values('simscore', ascending = False)


    return dfsort


In [21]:
# Getting Phrase Embeddings from Word Embeddings

def getWordVec(word,model):
        samp=model['users']
        vec=[0]*len(samp);
        try:
                vec=model[word]
        except:
                vec=[0]*len(samp)
        return (vec)


def getPhraseEmbedding(phrase,embeddingmodel):
                       
        samp=getWordVec('users', embeddingmodel)
        vec=np.array([0]*len(samp));
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+np.array(getWordVec(word,embeddingmodel))
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)

In [25]:
%%time
#With wb_word2vec_model



sent_embeddings=[]
for sent in sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,wb_word2vec_model))

    

question_embedding=getPhraseEmbedding(query,wb_word2vec_model)

dff = get_cosim(question_embedding,sent_embeddings).head(5)
dff

Wall time: 37.4 s


,id,simscore,result
48790,48790,0.822549,sap cosmos error
45768,45768,0.793013,sap error urgent
62411,62411,0.793013,urgent error in sap
55058,55058,0.777312,clscritical cosmos error sap
13069,13069,0.771350,sap error


In [26]:
for i in dff.index:
    print(i, dff['result'][i])

    

48790 sap cosmos error
45768 sap  error urgent
62411 urgent error in sap
55058 clscritical cosmos error   sap
13069 sap error
